In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import json
from aiida import load_profile
from aiida.orm import StructureData, load_code, Float, load_node
from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData
from aiida.engine import submit
from ase.atoms import Atoms
from ase.visualize import view
from aiida.plugins import DataFactory 
from pathlib import Path
from aiida_quantumespresso.common.types import SpinType
from aiida.orm import Int
from copy import deepcopy
import os

load_profile()

In [79]:

# ! Currently, for whatever reason, the codes are duplicated
# pw_code = load_code('qe-dev-pw@lumi-small')
# hp_code = load_code('qe-dev-hp@lumi-small')
pw_code = load_code(2182)
hp_code = load_code(2183)

hubbard_data = load_node(3075)
print(hubbard_data.get_ase())
print(hubbard_data.get_quantum_espresso_hubbard_card())

Atoms(symbols='Mn4O16P4Li4', pbc=True, cell=[[3.4378862545423e-06, 0.0, -4.740084535909], [0.0, -6.0917501186611, 0.0], [-10.474576328827, 0.0, -4.7800040115709e-06]], masses=...)
HUBBARD	ortho-atomic
 U	Mn-3d	4.5618
 V	Mn-3d	O-2p	1	518	0.0001
 V	Mn-3d	O-2p	1	601	0.0001
 V	Mn-3d	O-2p	1	373	0.0001
 V	Mn-3d	O-2p	1	621	0.0001
 V	Mn-3d	O-2p	1	326	0.0001
 V	Mn-3d	O-2p	1	17	0.0001
 V	Mn-3d	O-2p	1	596	0.0001
 V	Mn-3d	O-2p	2	15	0.0001
 V	Mn-3d	O-2p	2	324	0.0001
 V	Mn-3d	O-2p	2	12	0.0001
 V	Mn-3d	O-2p	2	8	0.0001
 V	Mn-3d	O-2p	2	19	0.0001
 V	Mn-3d	O-2p	2	328	0.0001
 V	Mn-3d	O-2p	2	5	0.0001
 V	Mn-3d	O-2p	3	18	0.0001
 V	Mn-3d	O-2p	3	17	0.0001
 V	Mn-3d	O-2p	3	11	0.0001
 V	Mn-3d	O-2p	3	371	0.0001
 V	Mn-3d	O-2p	3	378	0.0001
 V	Mn-3d	O-2p	3	377	0.0001
 V	Mn-3d	O-2p	3	6	0.0001
 V	Mn-3d	O-2p	4	20	0.0001
 V	Mn-3d	O-2p	4	19	0.0001
 V	Mn-3d	O-2p	4	626	0.0001
 V	Mn-3d	O-2p	4	6	0.0001
 V	Mn-3d	O-2p	4	632	0.0001
 V	Mn-3d	O-2p	4	631	0.0001
 V	Mn-3d	O-2p	4	7	0.0001



***
## Set up workchain with all of Iurii's settings

In [ ]:
builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'olivine_iurii_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

scf_dict = builder.scf.pw.parameters.get_dict()
relax_dict = builder.relax.base.pw.parameters.get_dict()
hubbard_dict = builder.hubbard.hp.parameters.get_dict()

# print(json.dumps(scf_dict, sort_keys=False, indent=4))
# print(json.dumps(relax_dict, sort_keys=False, indent=4))
# print(json.dumps(hubbard_dict, sort_keys=False, indent=4))

In [ ]:
# proper_olivine_builder = deepcopy(builder)
# proper_olivine_submit = submit(proper_olivine_builder)
# proper_olivine_submit_pk = proper_olivine_submit.pk
proper_olivine_submit_pk = 4282

In [ ]:
!verdi process status 4282

***
## Change only force convergence threshold in the optimization to see if `reconstruction problem` persists

In [ ]:
only_force_thr_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'only_force_thr.yaml')),
    spin_type=SpinType.COLLINEAR,
)

# only_force_thr_builder = submit(only_force_thr_builder)
# only_force_thr_submit_pk = only_force_thr_builder.pk
# print(only_force_thr_submit_pk)

In [40]:
# only_force_thr_submit_pk = 4376
!verdi process status 4376

SelfConsistentHubbardWorkChain<4376> Finished [402] [1:while_(should_run_iteration)(1:if_(should_run_relax)(1:inspect_relax))]
    ├── PwBaseWorkChain<4379> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<4380> Finished [0]
    │   └── PwCalculation<4384> Finished [0]
    ├── PwBaseWorkChain<4391> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<4392> Finished [0]
    │   └── PwCalculation<4396> Finished [0]
    ├── HpWorkChain<4402> Finished [0] [2:results]
    │   └── HpBaseWorkChain<4404> Finished [0] [3:results]
    │       └── HpCalculation<4406> Finished [0]
    ├── structure_relabel_kinds<4414> Finished [0]
    └── PwRelaxWorkChain<4418> Finished [401] [1:while_(should_run_relax)(1:inspect_relax)]
        └── PwBaseWorkChain<4421> Finished [401] [4:results]
            ├── create_kpoints_from_distance<4422> Finished [0]
            ├── PwCalculation<4426> Finished [400]
            ├── PwCalculation<4440> Finished [400]
            ├── PwCalcul

***
## True single-shot (only scf + hp), with default workchain settings

In [ ]:
from aiida.orm import Bool, Int

true_singleshot_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

_ = true_singleshot_builder.pop("relax", None)
true_singleshot_builder.meta_convergence = Bool(False)
true_singleshot_builder.max_iterations = Int(1)

# true_singleshot_submit = submit(true_singleshot_builder)
# print(true_singleshot_submit.pk)



In [ ]:
true_singleshot_submit_pk = 4774
# !verdi process status 4774
true_ss_workchain_node = load_node(true_singleshot_submit_pk)
print(true_ss_workchain_node.mtime - true_ss_workchain_node.ctime)

3:26:17.840074


***
## Default workchain settings, no relaxation


In [33]:
default_norelax_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'basically_empty_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

# _ = default_norelax_builder.pop("relax", None)
# default_norelax_submit = submit(default_norelax_builder)
# print(default_norelax_submit.pk)


In [37]:
default_norelax_submit_pk = 4512
# !verdi process status 4512
default_workchain_node = load_node(default_norelax_submit_pk)
print(default_workchain_node.mtime - default_workchain_node.ctime)

10:21:44.576013


***
## All of Iurii's settings, but no relaxation

In [ ]:
iurii_norelax_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'olivine_iurii_overrides.yaml')),
    spin_type=SpinType.COLLINEAR,
)

_ = iurii_norelax_builder.pop("relax", None)
# iurii_norelax_builder = submit(iurii_norelax_builder)
# print(iurii_norelax_builder.pk)

In [ ]:
iurii_norelax_builder_pk = 4486 
# !verdi process status 4486
iurii_workchain_node = load_node(iurii_norelax_builder_pk)
print(iurii_workchain_node.mtime - iurii_workchain_node.ctime)

1 day, 1:09:15.669150


***
## Compare results of true SS, workchain defaults, and Iurii's settings

In [ ]:
true_ss_output_hubbard = load_node(4813)
default_output_hubbard = load_node(4699)
iurii_output_hubbard = load_node(4822)

In [80]:
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData
from pprint import pprint
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
# type(true_ss_output_hubbard)

true_ss_hubbard_array = np.array([el[4] for el in true_ss_output_hubbard.hubbard_parameters])
default_hubbard_array = np.array([el[4] for el in default_output_hubbard.hubbard_parameters])
iurii_hubbard_array = np.array([el[4] for el in iurii_output_hubbard.hubbard_parameters])

pprint(mae(true_ss_hubbard_array, default_hubbard_array))
pprint(mae(true_ss_hubbard_array, iurii_hubbard_array))
pprint(mae(default_hubbard_array, iurii_hubbard_array))

print(true_ss_output_hubbard.get_quantum_espresso_hubbard_card())
print(default_output_hubbard.get_quantum_espresso_hubbard_card())
print(iurii_output_hubbard.get_quantum_espresso_hubbard_card())

pprint(true_ss_hubbard_array)
pprint(default_hubbard_array)
pprint(iurii_hubbard_array)

print(np.abs(true_ss_hubbard_array-iurii_hubbard_array))
print(np.abs(default_hubbard_array-iurii_hubbard_array))


0.03624285714285724
0.0357428571428572
0.0007857142857142943
HUBBARD	{ortho-atomic}
 U	Mn0-3d	4.2809
 V	Mn0-3d	O1-2p	1	518	0.7464
 V	Mn0-3d	O1-2p	1	601	0.7464
 V	Mn0-3d	O1-2p	1	373	0.7746
 V	Mn0-3d	O1-2p	1	621	0.518
 V	Mn0-3d	O1-2p	1	17	0.3979
 V	Mn0-3d	O1-2p	1	326	0.3979
 V	Mn0-3d	O1-2p	2	15	0.7464
 V	Mn0-3d	O1-2p	2	324	0.7464
 V	Mn0-3d	O1-2p	2	12	0.7746
 V	Mn0-3d	O1-2p	2	8	0.518
 V	Mn0-3d	O1-2p	2	19	0.3979
 V	Mn0-3d	O1-2p	2	328	0.3979
 V	Mn0-3d	O1-2p	3	17	0.7464
 V	Mn0-3d	O1-2p	3	18	0.7464
 V	Mn0-3d	O1-2p	3	11	0.7746
 V	Mn0-3d	O1-2p	3	371	0.518
 V	Mn0-3d	O1-2p	3	378	0.3979
 V	Mn0-3d	O1-2p	3	377	0.3979
 V	Mn0-3d	O1-2p	4	20	0.7464
 V	Mn0-3d	O1-2p	4	19	0.7464
 V	Mn0-3d	O1-2p	4	626	0.7746
 V	Mn0-3d	O1-2p	4	6	0.518
 V	Mn0-3d	O1-2p	4	631	0.3979
 V	Mn0-3d	O1-2p	4	632	0.3979

HUBBARD	{ortho-atomic}
 U	Mn0-3d	4.4248
 V	Mn0-3d	O1-2p	1	518	0.7728
 V	Mn0-3d	O1-2p	1	601	0.7728
 V	Mn0-3d	O1-2p	1	373	0.8021
 V	Mn0-3d	O1-2p	1	621	0.5343
 V	Mn0-3d	O1-2p	1	17	0.4045
 V	Mn0-3d	O1-2p	1	326	0.4045
 V	Mn0

In [82]:
pprint(true_ss_hubbard_array-default_hubbard_array)
pprint(default_hubbard_array-iurii_hubbard_array)

array([-0.1439, -0.0264, -0.0264, -0.0275, -0.0163, -0.0066, -0.0066,
       -0.1439, -0.0264, -0.0264, -0.0275, -0.0163, -0.0066, -0.0066,
       -0.1439, -0.0264, -0.0264, -0.0275, -0.0163, -0.0066, -0.0066,
       -0.1439, -0.0264, -0.0264, -0.0275, -0.0163, -0.0066, -0.0066])
array([ 4.5e-03, -1.0e-04, -1.0e-04, -2.0e-04, -2.0e-04, -2.0e-04,
       -2.0e-04,  4.5e-03, -1.0e-04, -1.0e-04, -2.0e-04, -2.0e-04,
       -2.0e-04, -2.0e-04,  4.5e-03, -1.0e-04, -1.0e-04, -2.0e-04,
       -2.0e-04, -2.0e-04, -2.0e-04,  4.5e-03, -1.0e-04, -1.0e-04,
       -2.0e-04, -2.0e-04, -2.0e-04, -2.0e-04])


***
## Set proper AFM magnetic ordering

In [ ]:
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

# print(hubbard_data.get_quantum_espresso_hubbard_card())
# hubbardstructuredata = HubbardStructureData(hubbard_data)
print(type(hubbard_data))

# hubbard_data.

In [ ]:

afm_magn_builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'only_force_thr.yaml')),
    spin_type=SpinType.COLLINEAR,
)


# afm_magn_builder = submit(afm_magn_builder)
# only_force_thr_submit_pk = afm_magn_builder.pk
# print(only_force_thr_submit_pk)
# only_force_thr_submit_pk = 4376
# !verdi process report 4376


In [ ]:

import pandas as pd
from project_settings import project_dir
from aiida.orm import StructureData

fully_lithiated_df = pd.read_pickle(os.path.join(project_dir, 'data', 'fully_lithiated_df.pkl'))
fully_lithiated_df['chem_formula'].values
mn_olivine_ase = fully_lithiated_df['ase_in'].values[2]

sorting_dict = {
    'Mn': 0,
    'O': 1,
    'P': 2,
    'Li': 3,
}

# ! Does not take into account initial magmoms for now
mn_olivine_ase_ordered = Atoms(sorted(mn_olivine_ase, key=lambda x: sorting_dict[x.symbol]))
mn_olivine_ase_ordered.set_cell(mn_olivine_ase.get_cell())
mn_olivine_structuredata = StructureData(ase=mn_olivine_ase_ordered)
StructureData

***
## Copied from Marnik's notebook

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the olivine structure!

In [ ]:
hubbard_data_inv = HubbardStructureData(structure=mn_olivine_aiida)

In [ ]:
# hubbard_data.reorder_atoms()

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [ ]:
# Taken from parameters.in of:
# /home/jgeiger/projects/bat_uv_ml/data/olivine_iurii/LixMnPO4/Li1.00/DFT_plus_UV/9_PDOS/LMPO.scf.1.in
hubbard_data_inv.initialize_onsites_hubbard('Mn', '3d', 4.5618)

Here how it is stored in the class.

In [ ]:
hubbard_data_inv.hubbard_parameters

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [ ]:
hubbard_data_inv.initialize_intersites_hubbard('Mn', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 
# ! Ti has 6 oxygen neighbors in BaTiO3. Similarly, Mn has 6 oxygen neighbors in LMPO (olivine). So, we need to use N+1 in the call to initialize_intersites_hubbard, as the value is used for list slicing, like [:number_of_neighbours], which is exclusive of the last index.

The parameters are saved in the property `hubbard_parameters` as a list.

In [ ]:
hubbard_data_inv.hubbard_parameters

In [ ]:
print(hubbard_data_inv.get_quantum_espresso_hubbard_card())
print(hubbard_data_inv.get_quantum_espresso_hubbard_parameters())

In [ ]:
print(type(mn_olivine_aiida))
print(mn_olivine_aiida.get_ase().get_chemical_symbols())
from aiida.orm.nodes.data.structure import StructureData

In [ ]:
hubbard_data_inv.store()
hubbard_data_inv.pk

In [ ]:
inverted_hubbard_data_pk = 3015